In [2]:
import h5py
import napari
import numpy as np
import pandas as pd
from abbott.h5_files import *
from abbott.conversions import *
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()

In [3]:
from skimage.measure import label, regionprops

def select_largest_object(bw_img):
    lbl_img = label(bw_img)
    props = regionprops(lbl_img)
    if not props:  # sometimes we don't have ANY mask for a timepoint which would throw an error in the next step if we don't check for it
        return np.zeros(bw_img.shape, dtype=np.int32)  # Return an integer type array
    lbl = [p.label for p in props][np.argmax([p.area for p in props])]
    return (lbl_img == lbl).astype(np.int32)  # Convert the output to integer type


In [4]:
from skimage.measure import label, regionprops
import shutil


#fld= "Q:\carspi\Test"
#trash_fld= "Q:\carspi\Test_trash"

def keep_in_frame(fld, trash_fld, tolerance):

    files_to_move = []

    for fn in fld.glob('*.h5'):
        with h5py.File(fn) as f:
            print('processing {}...'.format(fn.name))

            mask_cleanup_corrected = f['mask_cleanup_corrected/1'][...]
            timepoints = mask_cleanup_corrected.shape[0]
            
            mask = label(mask_cleanup_corrected)  
            mask = select_largest_object(mask)
            props = regionprops(mask)

            area_tmin1 = props[0].area
           
            for t in range(timepoints):
               
                if not props: #check if there is a region so that it doesn't create problems later
                    #if there is an area with no mask, it's OOF
                    #save embryo to trash folder
                    files_to_move.append(fn)
                    break

                area_t = props[0].area #measure area at timepoint t

                tol = area_t * tolerance

                print('processing timepoint', t, 'area', area_t)
                
                if area_t <= area_tmin1 - tol:

                    files_to_move.append(fn)
                    break

                area_tmin1 = area_t #update area to that of t for next comparison
    
    for fn in files_to_move:
        shutil.move(fn, trash_fld)
                  

In [ ]:
fld=Path(r"Q:\carspi\Test")
trash_fld=Path(r"Q:\carspi\Test_trash")

keep_in_frame(fld, trash_fld, tolerance=0.1)